<a href="https://colab.research.google.com/github/mihir70/Mask_Vs_No_Mask/blob/main/mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import PIL
import cv2
from random import shuffle
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [3]:
from glob import glob
non=list(glob('/content/drive/MyDrive/Mask_vs_No_Mask/data/anotherdataset/nomask/*.png'))
non=non[:2000]
len(non)

2000

In [4]:
mask=list(glob('/content/drive/MyDrive/Mask_vs_No_Mask/data/anotherdataset/mask/*.png'))
mask=mask[:2000]

In [8]:
non_im=[]
for i in non:
  img=cv2.imread(i)
  img=cv2.resize(img,(128,128))
  non_im.append(img)



In [10]:
len(non_im)

2000

In [11]:
mask_im=[]
for i in mask:
  img=cv2.imread(i)
  img=cv2.resize(img,(128,128))
  mask_im.append(img)


In [12]:
len(mask_im)

2000

In [13]:
img_rows=128
img_cols=128
non_im=np.array(non_im)/255.0
mask_im=np.array(mask_im)/255.0

non_im = np.reshape(non_im, (non_im.shape[0], img_rows, img_cols, 3))
mask_im = np.reshape(mask_im, (mask_im.shape[0], img_rows, img_cols, 3))

In [14]:
type(non_im)
print(non_im.shape)
print(non_im[1])

(2000, 128, 128, 3)
[[[0.45098039 0.45490196 0.45098039]
  [0.45490196 0.45490196 0.45098039]
  [0.45098039 0.45098039 0.44705882]
  ...
  [0.45490196 0.45882353 0.45098039]
  [0.45490196 0.45882353 0.45098039]
  [0.45490196 0.45882353 0.44705882]]

 [[0.98039216 0.98431373 0.97647059]
  [0.98823529 0.99215686 0.98431373]
  [0.98431373 0.98823529 0.98431373]
  ...
  [0.99607843 1.         0.99215686]
  [0.99607843 1.         0.99215686]
  [0.99607843 1.         0.98823529]]

 [[0.98039216 0.98431373 0.97647059]
  [0.97647059 0.98431373 0.97254902]
  [0.98039216 0.98431373 0.97647059]
  ...
  [0.99215686 0.99607843 0.98823529]
  [0.99215686 0.99607843 0.99215686]
  [0.99215686 0.99607843 0.98431373]]

 ...

 [[0.36078431 0.28235294 0.15686275]
  [0.67058824 0.60784314 0.47843137]
  [0.83529412 0.78823529 0.66666667]
  ...
  [0.58039216 0.68235294 0.77647059]
  [0.55686275 0.65490196 0.74901961]
  [0.54901961 0.64313725 0.7372549 ]]

 [[0.4627451  0.38823529 0.25490196]
  [0.78823529 0.7

In [15]:
data=np.concatenate((non_im,mask_im),axis=0)
no=np.empty(2000)
no.fill(0) #0 for no mask
ye=np.empty(2000)
ye.fill(1)
label=np.concatenate((no,ye))

In [ ]:
data[0]

array([[[ 71, 108, 116],
        [ 62,  98, 106],
        [ 52,  88,  95],
        ...,
        [180, 198, 199],
        [178, 196, 197],
        [179, 197, 198]],

       [[ 76, 112, 121],
        [ 65, 101, 109],
        [ 52,  86,  93],
        ...,
        [180, 198, 199],
        [179, 197, 198],
        [179, 197, 198]],

       [[ 76, 110, 119],
        [ 69, 103, 110],
        [ 57,  91,  96],
        ...,
        [181, 199, 200],
        [179, 197, 198],
        [179, 197, 198]],

       ...,

       [[239, 250, 240],
        [210, 225, 215],
        [162, 180, 171],
        ...,
        [ 28,  25,  21],
        [ 28,  25,  20],
        [ 28,  25,  20]],

       [[241, 252, 242],
        [212, 226, 216],
        [160, 177, 168],
        ...,
        [ 27,  24,  20],
        [ 27,  24,  19],
        [ 27,  24,  19]],

       [[241, 253, 243],
        [212, 226, 216],
        [157, 172, 164],
        ...,
        [ 27,  24,  20],
        [ 26,  23,  19],
        [ 26,  23,  18]]

In [16]:
lb = LabelBinarizer()
label = lb.fit_transform(label)
label = to_categorical(label)
label = np.array(label)

In [17]:
(X_train, X_test, y_train, y_test) = train_test_split(
    data, label, test_size=0.3, random_state=0)


In [ ]:
len(X_train)

3418

In [18]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [55]:
model = Sequential([
    layers.Conv2D(128, (3, 3), padding='same', activation='relu',
                  input_shape=(img_rows, img_cols, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(2, activation='softmax')
])

In [56]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [57]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 128, 128, 128)     3584      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 64, 64, 64)        73792     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 16, 16, 32)       

In [58]:
model.fit(X_train,y_train,epochs=20)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


88/88 [==============================] - 4s 35ms/step - loss: 0.3961 - accuracy: 0.8150
Epoch 2/20
88/88 [==============================] - 3s 35ms/step - loss: 0.2018 - accuracy: 0.9254
Epoch 3/20
88/88 [==============================] - 3s 34ms/step - loss: 0.1605 - accuracy: 0.9400
Epoch 4/20
88/88 [==============================] - 3s 34ms/step - loss: 0.1233 - accuracy: 0.9554
Epoch 5/20
88/88 [==============================] - 3s 35ms/step - loss: 0.0955 - accuracy: 0.9646
Epoch 6/20
88/88 [==============================] - 3s 35ms/step - loss: 0.1118 - accuracy: 0.9575
Epoch 7/20
88/88 [==============================] - 3s 35ms/step - loss: 0.0897 - accuracy: 0.9689
Epoch 8/20
88/88 [==============================] - 3s 35ms/step - loss: 0.0695 - accuracy: 0.9750
Epoch 9/20
88/88 [==============================] - 3s 35ms/step - loss: 0.0609 - accuracy: 0.9807
Epoch 10/20
88/88 [==============================] - 3s 35ms/step - loss: 0.0491 - accuracy: 0.9846
Epoch 11/20
88/88 [=

In [59]:
prediction=model.predict(X_test)
y_pred=lb.inverse_transform(prediction)
y_test=lb.inverse_transform(y_test)


In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.9766666666666667

In [62]:
model.save('/content/drive/MyDrive/Mask_vs_No_Mask/model')


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Mask_vs_No_Mask/model/assets
